In [187]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso,Ridge





In [2]:
df = pd.read_csv("data/output/output_complete.csv",sep=";")



In [23]:
dfY=df.iloc[:,1:]
dfS=df.drop(["Year"],axis=1)


# Spatial Cross validation

In [252]:
train, validation = train_test_split(dfS, test_size=0.2)

df_gt = pd.read_csv('data/BRFSS/2011_AA.csv', sep=";")
states = df_gt["LocationDesc"][0:51]


In [237]:
mseLM = []
mseLasso = []
mseRidge = []
# LOO CrossValidation
for state in states:
    trainK = train.loc[train["State"] == state]
    testK = train.drop(trainK.index)

    #LM
    lm = linear_model.LinearRegression()
    model = lm.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predictions = lm.predict(testK.iloc[:,1:-1])
    mseLM.append(np.mean((predictions - testK.iloc[:,-1])**2))


    #Lasso
    trainKnum = trainK.apply(pd.to_numeric, errors='coerce')
    testKnum = testK.apply(pd.to_numeric, errors='coerce')
    trainKnum.fillna(0, inplace=True)
    testKnum.fillna(0, inplace=True)
    
    lassoReg = Lasso()
    lassoReg.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predLasso = lassoReg.predict(testK.iloc[:,1:-1])
    mseLasso.append(np.mean((predLasso - testK.iloc[:,-1])**2))

    #Ridge
    ridgeReg = Ridge()
    ridgeReg.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predRidge = ridgeReg.predict(testK.iloc[:,1:-1])
    mseRidge.append(np.mean((predRidge - testK.iloc[:,-1])**2))

lmod = np.mean(mseLM)
las = np.mean(mseLasso)
ridge = np.mean(mseRidge)


print(" # mean LM:",np.mean(mseLM))
print(" # mean Lasso:",np.mean(mseLasso))
print(" # mean Ridge:",np.mean(mseRidge))
print(" # mean time cv: ", np.mean([np.mean(mseLM),np.mean(mseLasso),np.mean(mseRidge)]))

 # mean LM: 0.42153830087
 # mean Lasso: 0.450547721089
 # mean Ridge: 0.389936875815
 # mean time cv:  0.420674299258


# Temporal cross validation

In [280]:
train, validation = train_test_split(dfY, test_size=0.2)



In [281]:
mseLM = []
mseLasso = []
mseRidge = []

for year in [2011,2012,2013,2014,2015,2016]:
    trainK = train.loc[train.Year == year]
    testK = train.drop(trainK.index)
    
    
    #lm
    lm = linear_model.LinearRegression()
    model = lm.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predictions = lm.predict(testK.iloc[:,1:-1])
    mseLM.append(np.mean((predictions - testK.iloc[:,-1])**2))


    
    #Lasso
    trainKnum = trainK.apply(pd.to_numeric, errors='coerce')
    testKnum = testK.apply(pd.to_numeric, errors='coerce')
    trainKnum.fillna(0, inplace=True)
    testKnum.fillna(0, inplace=True)
    
    lassoReg = Lasso(alpha=1)
    lassoReg.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predLasso = lassoReg.predict(testK.iloc[:,1:-1])
    mseLasso.append(np.mean((predLasso - testK.iloc[:,-1])**2))

    #Ridge
    ridgeReg = Ridge(alpha=1)
    ridgeReg.fit(trainK.iloc[:,1:-1],trainK.iloc[:,-1])
    predRidge = ridgeReg.predict(testK.iloc[:,1:-1])
    mseRidge.append(np.mean((predRidge - testK.iloc[:,-1])**2))

lm2 = np.mean(mseLM)
las2 = np.mean(mseLasso)
ridge2 = np.mean(mseRidge)



    

print(" # mean LM:",np.mean(mseLM))
print(" # mean Lasso:",np.mean(mseLasso))
print(" # mean Ridge:",np.mean(mseRidge))
print(" # mean time cv: ", np.mean([np.mean(mseLM),np.mean(mseLasso),np.mean(mseRidge)]))

 # mean LM: 0.273681027602
 # mean Lasso: 0.228940466016
 # mean Ridge: 0.274595643301
 # mean time cv:  0.259072378973


In [282]:
print("lm:", np.mean([lmod,lm2]))
print("lasso:", np.mean([las,las2]))
print("ridge:", np.mean([ridge,ridge2]))


lm: 0.347609664236
lasso: 0.339744093553
ridge: 0.332266259558


In [286]:
# lasso best model
from sklearn.linear_model import Lasso
lassoReg = Lasso(alpha=0.2)

lassoReg.fit(train.iloc[:,1:-1],train.iloc[:,-1])

pred2 = lassoReg.predict(validation.iloc[:,1:-1])

# calculating mse

mse = np.mean((pred2 - validation.iloc[:,-1])**2)

print("mse: ",mse)
print("R squared: ",lassoReg.score(validation.iloc[:,1:-1],validation.iloc[:,-1]))



mse:  0.14110554865448396
R squared:  0.239566334832


# Add to your model census features at the US state level (e.g., income data) and discuss how adding them impacts the predictive performance of the model.



In [276]:
income= pd.read_csv("data/Poverty/income.csv", sep=";")

In [277]:
dfs=pd.concat([dfS,income.Income],axis=1)
X = dfs.drop("Data_Value", axis = 1) 
y = dfs.Data_Value
X = X.apply(pd.to_numeric, errors='coerce')
y = y.apply(pd.to_numeric, errors='coerce')
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)

In [278]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [287]:
from sklearn.linear_model import Lasso
lassoReg = Lasso(alpha=0.2)

lassoReg.fit(X_train,y_train)

pred2 = lassoReg.predict(X_test)

# calculating mse

mse = np.mean((pred2 - y_test)**2)

print("mse :",mse)


print("R squared: ",lassoReg.score(X_test,y_test))



mse : 0.125246673343142
R squared:  0.377058130858


### As a result we get better results (mse reduction from 0.14 to 0.12) adding the "income" variable.